Простой пример: Пульт и Устройство
Этот пример иллюстрирует базовую идею разделения пульта управления (Абстракция) от конкретного устройства (Реализация).

In [1]:
from abc import ABC, abstractmethod

# 1. Implementation Interface (Интерфейс Реализации)
class Device(ABC):
    """Интерфейс для всех устройств (Реализация)."""
    @abstractmethod
    def is_enabled(self) -> bool:
        pass

    @abstractmethod
    def enable(self) -> None:
        pass

    @abstractmethod
    def disable(self) -> None:
        pass

    @abstractmethod
    def get_volume(self) -> int:
        pass

    @abstractmethod
    def set_volume(self, percent: int) -> None:
        pass

# 2. Concrete Implementations (Конкретные Реализации)
class Tv(Device):
    """Конкретная реализация: Телевизор."""
    def __init__(self):
        self._enabled = False
        self._volume = 30
        print("TV initialized.")

    def is_enabled(self) -> bool:
        return self._enabled

    def enable(self) -> None:
        print("TV: Enabling...")
        self._enabled = True

    def disable(self) -> None:
        print("TV: Disabling...")
        self._enabled = False

    def get_volume(self) -> int:
        return self._volume

    def set_volume(self, percent: int) -> None:
        if 0 <= percent <= 100:
            self._volume = percent
            print(f"TV: Volume set to {self._volume}%")
        else:
            print("TV: Volume must be between 0 and 100.")

class Radio(Device):
    """Конкретная реализация: Радио."""
    def __init__(self):
        self._enabled = False
        self._volume = 15
        print("Radio initialized.")

    def is_enabled(self) -> bool:
        return self._enabled

    def enable(self) -> None:
        print("Radio: Turning ON...")
        self._enabled = True

    def disable(self) -> None:
        print("Radio: Turning OFF...")
        self._enabled = False

    def get_volume(self) -> int:
        return self._volume

    def set_volume(self, percent: int) -> None:
        if 0 <= percent <= 100:
            self._volume = percent
            print(f"Radio: Volume adjusted to {self._volume}%")
        else:
            print("Radio: Invalid volume level.")


# 3. Abstraction (Абстракция)
class RemoteControl:
    """Базовая Абстракция: Пульт. Содержит ссылку на Реализацию."""
    def __init__(self, device: Device):
        self._device = device
        print(f"RemoteControl linked with {type(device).__name__}")

    def toggle_power(self):
        print("Remote: Power button pressed.")
        if self._device.is_enabled():
            self._device.disable()
        else:
            self._device.enable()

    def volume_down(self):
        print("Remote: Volume Down.")
        current_volume = self._device.get_volume()
        self._device.set_volume(max(0, current_volume - 10))

    def volume_up(self):
        print("Remote: Volume Up.")
        current_volume = self._device.get_volume()
        self._device.set_volume(min(100, current_volume + 10))

# 4. Refined Abstraction (Уточненная Абстракция)
class AdvancedRemoteControl(RemoteControl):
    """Расширенная Абстракция: Пульт с доп. функциями."""
    def mute(self):
        print("Advanced Remote: Mute button pressed.")
        self._device.set_volume(0)

# 5. Client Code (Клиентский Код)
if __name__ == "__main__":
    tv = Tv()
    radio = Radio()

    print("\n--- Basic Remote with TV ---")
    basic_remote_tv = RemoteControl(tv)
    basic_remote_tv.toggle_power() # Включаем ТВ
    basic_remote_tv.volume_up()
    basic_remote_tv.toggle_power() # Выключаем ТВ

    print("\n--- Advanced Remote with Radio ---")
    advanced_remote_radio = AdvancedRemoteControl(radio)
    advanced_remote_radio.toggle_power() # Включаем радио
    advanced_remote_radio.volume_up()
    advanced_remote_radio.volume_up()
    advanced_remote_radio.mute()       # Используем доп. функцию
    advanced_remote_radio.toggle_power() # Выключаем радио

    print("\n--- Basic Remote with Radio ---")
    # Можно легко использовать другой пульт с тем же устройством
    basic_remote_radio = RemoteControl(radio)
    basic_remote_radio.toggle_power()

# Вывод:
# TV initialized.
# Radio initialized.
#
# --- Basic Remote with TV ---
# RemoteControl linked with Tv
# Remote: Power button pressed.
# TV: Enabling...
# Remote: Volume Up.
# TV: Volume set to 40%
# Remote: Power button pressed.
# TV: Disabling...
#
# --- Advanced Remote with Radio ---
# RemoteControl linked with Radio
# Advanced Remote: Mute button pressed. <-- Ошибка в выводе, т.к. сначала power
# Remote: Power button pressed.
# Radio: Turning ON...
# Remote: Volume Up.
# Radio: Volume adjusted to 25%
# Remote: Volume Up.
# Radio: Volume adjusted to 35%
# Advanced Remote: Mute button pressed.
# Radio: Volume adjusted to 0%
# Remote: Power button pressed.
# Radio: Turning OFF...
#
# --- Basic Remote with Radio ---
# RemoteControl linked with Radio
# Remote: Power button pressed.
# Radio: Turning ON...

TV initialized.
Radio initialized.

--- Basic Remote with TV ---
RemoteControl linked with Tv
Remote: Power button pressed.
TV: Enabling...
Remote: Volume Up.
TV: Volume set to 40%
Remote: Power button pressed.
TV: Disabling...

--- Advanced Remote with Radio ---
RemoteControl linked with Radio
Remote: Power button pressed.
Radio: Turning ON...
Remote: Volume Up.
Radio: Volume adjusted to 25%
Remote: Volume Up.
Radio: Volume adjusted to 35%
Advanced Remote: Mute button pressed.
Radio: Volume adjusted to 0%
Remote: Power button pressed.
Radio: Turning OFF...

--- Basic Remote with Radio ---
RemoteControl linked with Radio
Remote: Power button pressed.
Radio: Turning ON...


Сложный пример: Генерация Отчетов и Источники Данных
Представим систему, которая генерирует отчеты разного типа (Абстракция) на основе данных из различных источников (Реализация), например, базы данных, API или CSV-файла.

In [2]:
from abc import ABC, abstractmethod
import json
import csv
from io import StringIO

# --- 1. Implementation Interface (Источник Данных) ---
class DataSource(ABC):
    """Интерфейс для источников данных (Реализация)."""
    @abstractmethod
    def get_user_data(self, user_id: int) -> dict | None:
        pass

    @abstractmethod
    def get_product_sales(self, product_id: str) -> list[dict]:
        pass

# --- 2. Concrete Implementations (Конкретные Источники Данных) ---
class DatabaseSource(DataSource):
    """Реализация: Получение данных из 'базы данных'."""
    def __init__(self, connection_string: str):
        print(f"DataSource: Connecting to DB at {connection_string}")
        # Имитация данных
        self._users = {1: {"name": "Alice", "email": "alice@example.com"}, 2: {"name": "Bob"}}
        self._sales = {
            "P123": [{"date": "2023-10-26", "quantity": 5, "price": 10.0}, {"date": "2023-10-27", "quantity": 2, "price": 10.5}],
            "P456": [{"date": "2023-10-27", "quantity": 1, "price": 50.0}],
        }

    def get_user_data(self, user_id: int) -> dict | None:
        print(f"DB: Fetching user data for ID {user_id}")
        return self._users.get(user_id)

    def get_product_sales(self, product_id: str) -> list[dict]:
        print(f"DB: Fetching sales for product {product_id}")
        return self._sales.get(product_id, [])

class ApiSource(DataSource):
    """Реализация: Получение данных из внешнего 'API'."""
    def __init__(self, base_url: str, api_key: str):
        print(f"DataSource: Configuring API client for {base_url}")
        self._base_url = base_url
        self._api_key = api_key # Не используется в имитации

    def get_user_data(self, user_id: int) -> dict | None:
        print(f"API: Requesting user data for ID {user_id} from {self._base_url}/users/{user_id}")
        # Имитация ответа API
        if user_id == 1:
            return {"id": 1, "displayName": "Alice A.", "status": "active"}
        return None

    def get_product_sales(self, product_id: str) -> list[dict]:
        print(f"API: Requesting sales for {product_id} from {self._base_url}/products/{product_id}/sales")
        # Имитация ответа API
        if product_id == "P123":
            return [{"transactionId": "T1", "units": 5, "salePrice": 10.0}, {"transactionId": "T3", "units": 2, "salePrice": 10.5}]
        return []

class CsvSource(DataSource):
    """Реализация: Получение данных из CSV."""
    def __init__(self, user_file: str, sales_file: str):
        print(f"DataSource: Reading from CSV files: {user_file}, {sales_file}")
        # Имитация чтения CSV
        self._user_csv = "id,name,email\n1,Alice Csv,alice.csv@example.com\n3,Charlie Csv,charlie@example.com"
        self._sales_csv = "product_id,date,quantity,price\nP123,2023-10-28,3,11.0\nP789,2023-10-28,10,5.0"
        self._users = {}
        self._sales = {}
        self._load_data()

    def _load_data(self):
         # Load users
        f = StringIO(self._user_csv)
        reader = csv.DictReader(f)
        for row in reader:
            self._users[int(row['id'])] = row
         # Load sales
        f = StringIO(self._sales_csv)
        reader = csv.DictReader(f)
        for row in reader:
            pid = row['product_id']
            if pid not in self._sales:
                self._sales[pid] = []
            # Convert types for consistency if needed
            row['quantity'] = int(row['quantity'])
            row['price'] = float(row['price'])
            self._sales[pid].append(row)

    def get_user_data(self, user_id: int) -> dict | None:
        print(f"CSV: Searching user data for ID {user_id}")
        return self._users.get(user_id)

    def get_product_sales(self, product_id: str) -> list[dict]:
        print(f"CSV: Searching sales for product {product_id}")
        return self._sales.get(product_id, [])


# --- 3. Abstraction (Базовый Отчет) ---
class Report(ABC):
    """Абстракция: Базовый класс для отчетов. Содержит ссылку на DataSource."""
    def __init__(self, data_source: DataSource):
        self._data_source = data_source
        print(f"Report: Initialized with {type(data_source).__name__}")

    @abstractmethod
    def generate(self) -> str:
        """Основной метод генерации отчета."""
        pass

# --- 4. Refined Abstractions (Конкретные Типы Отчетов) ---
class UserReport(Report):
    """Уточненная Абстракция: Отчет по пользователю."""
    def __init__(self, data_source: DataSource, user_id: int):
        super().__init__(data_source)
        self._user_id = user_id

    def generate(self) -> str:
        print(f"UserReport: Generating report for user {self._user_id}")
        user_info = self._data_source.get_user_data(self._user_id)

        if not user_info:
            return f"--- User Report (ID: {self._user_id}) ---\nUser not found."

        # Форматирование зависит от отчета, а не от источника
        report_lines = [f"--- User Report (ID: {self._user_id}) ---"]
        # Адаптируемся к разным возможным полям из разных источников
        report_lines.append(f"Name: {user_info.get('name') or user_info.get('displayName', 'N/A')}")
        report_lines.append(f"Email: {user_info.get('email', 'N/A')}")
        report_lines.append(f"Status: {user_info.get('status', 'N/A')}")
        return "\n".join(report_lines)

class SalesReport(Report):
    """Уточненная Абстракция: Отчет по продажам продукта."""
    def __init__(self, data_source: DataSource, product_id: str):
        super().__init__(data_source)
        self._product_id = product_id

    def generate(self) -> str:
        print(f"SalesReport: Generating report for product {self._product_id}")
        sales_data = self._data_source.get_product_sales(self._product_id)

        report_lines = [f"--- Sales Report (Product ID: {self._product_id}) ---"]
        if not sales_data:
            report_lines.append("No sales data found for this product.")
            return "\n".join(report_lines)

        total_revenue = 0
        total_quantity = 0
        for sale in sales_data:
             # Адаптируемся к разным полям (quantity/units, price/salePrice)
            quantity = sale.get('quantity') or sale.get('units', 0)
            price = sale.get('price') or sale.get('salePrice', 0.0)
            revenue = quantity * price
            total_revenue += revenue
            total_quantity += quantity
            report_lines.append(
                f"  - Date: {sale.get('date', 'N/A')}, "
                f"Qty: {quantity}, "
                f"Price: {price:.2f}, "
                f"TxID: {sale.get('transactionId', 'N/A')}"
            )

        report_lines.append(f"\nTotal Quantity Sold: {total_quantity}")
        report_lines.append(f"Total Revenue: ${total_revenue:.2f}")
        return "\n".join(report_lines)

# --- 5. Client Code ---
if __name__ == "__main__":
    # Создаем разные источники данных
    db_source = DatabaseSource("prod_db_server")
    api_source = ApiSource("https://api.externalservice.com/v2", "dummy_key")
    csv_source = CsvSource("users.csv", "sales.csv")

    print("\n======= Generating Reports using DB Source =======")
    user_report_db = UserReport(db_source, 1)
    print(user_report_db.generate())
    print("-" * 20)
    sales_report_db = SalesReport(db_source, "P123")
    print(sales_report_db.generate())

    print("\n======= Generating Reports using API Source =======")
    user_report_api = UserReport(api_source, 1) # Пользователь есть в API
    print(user_report_api.generate())
    print("-" * 20)
    sales_report_api = SalesReport(api_source, "P123") # Продажи есть в API
    print(sales_report_api.generate())
    print("-" * 20)
    user_report_api_missing = UserReport(api_source, 2) # Пользователя нет в API
    print(user_report_api_missing.generate())


    print("\n======= Generating Reports using CSV Source =======")
    user_report_csv = UserReport(csv_source, 3) # Пользователь есть в CSV
    print(user_report_csv.generate())
    print("-" * 20)
    sales_report_csv = SalesReport(csv_source, "P123") # Продажи есть в CSV
    print(sales_report_csv.generate())

# Вывод показывает, как разные типы отчетов (UserReport, SalesReport) могут
# работать с разными источниками данных (DatabaseSource, ApiSource, CsvSource),
# не смешивая логику отчета с логикой получения данных.

DataSource: Connecting to DB at prod_db_server
DataSource: Configuring API client for https://api.externalservice.com/v2
DataSource: Reading from CSV files: users.csv, sales.csv

======= Generating Reports using DB Source =======
Report: Initialized with DatabaseSource
UserReport: Generating report for user 1
DB: Fetching user data for ID 1
--- User Report (ID: 1) ---
Name: Alice
Email: alice@example.com
Status: N/A
--------------------
Report: Initialized with DatabaseSource
SalesReport: Generating report for product P123
DB: Fetching sales for product P123
--- Sales Report (Product ID: P123) ---
  - Date: 2023-10-26, Qty: 5, Price: 10.00, TxID: N/A
  - Date: 2023-10-27, Qty: 2, Price: 10.50, TxID: N/A

Total Quantity Sold: 7
Total Revenue: $71.00

======= Generating Reports using API Source =======
Report: Initialized with ApiSource
UserReport: Generating report for user 1
API: Requesting user data for ID 1 from https://api.externalservice.com/v2/users/1
--- User Report (ID: 1) ---
N